# 案例：ChatGPT的API调用

本案例讲解如何结合网络爬虫以及API来实现更为复杂的逻辑。

案例参考了Lopez-Lira and Tang(2023)中的思想，即输入特定的prompt，让ChatGPT对输入的新闻标题进行情感分析，将其分为积极、消极和未知。参考Lopez-Lira and Tang(2023)中的prompt

本案例中使用的prompt为
```
忘掉之前所有的指令。假装你是一位金融专家。你是一位有荐股经验的金融专家。如果是好消息，请回答 "积极"；如果是坏消息，请回答 "消极"；如果是不确定的消息，请回答 "未知"。然后在下一行用一个简短的句子详细说明。
```

本案例中首先使用网络爬虫爬取了新浪财经首页的要闻标题内容，然后调用OpenAI提供的API来对爬取到的要闻标题进行情感分析，最后对调用的结果进行简单的展示。其中网络爬虫部分的代码来自于网络数据采集三部曲。

## 要闻标题爬取
首先导入必要的包

In [1]:
import requests # 导入库
import parsel
from pprint import pprint # 为了更好的打印字典
import openai
import os
from time import sleep

首先请求新浪财经的首页url，拿到对应的响应数据

In [ ]:
url = "https://finance.sina.com.cn/" # 请求的目标地址
res = requests.get(url) # 使用requests库来发送GET请求，获得响应

接着修改响应数据的编码，解决中文乱码的问题，然后使用`parsel`库解析响应数据提取要闻相关的数据

In [3]:
res.encoding = "utf-8" # 将编码设置为utf-8
page = parsel.Selector(res.text) # 将响应数据进行解析
outer_div = page.xpath("//div[@class='fin_tabs0_c0']") # 找 class值为fin_tabs0_c0的div节点
a_under_div = outer_div.xpath(".//a") # div下所有的a节点

data = list() # 声明一个列表来存储数据
for item in a_under_div: # 遍历每一个a节点，取其href和文本值
    news_title = item.xpath("./text()").extract_first() # 文本值作为要闻标题
    news_url = item.xpath("./@href").extract_first() # href值作为要闻url
    this_data = {
        "news_title" : news_title,
        "news_url" : news_url, 
    }
    data.append(this_data)
pprint(data)

[{'news_title': '经济日报:以暂停新股发行来救市不可取',
  'news_url': 'https://finance.sina.com.cn/jjxw/2023-08-17/doc-imzhmzqi3469850.shtml '},
 {'news_title': '股市转牛依靠印花税？机构：政策象征意义或大于资金面宽松意义',
  'news_url': 'https://finance.sina.com.cn/roll/2023-08-16/doc-imzhmkst2490734.shtml'},
 {'news_title': '减免证券交易印花税呼声渐高！分析人士称具备调整必要性和空间',
  'news_url': 'https://finance.sina.com.cn/stock/marketresearch/2023-08-17/doc-imzhnfwc6582020.shtml '},
 {'news_title': '美联储会议纪要暗示加息或将继续',
  'news_url': 'https://finance.sina.com.cn/stock/usstock/c/2023-08-17/doc-imzhmqyp3709471.shtml'},
 {'news_title': '通胀风险成联储纪要关键词',
  'news_url': 'https://finance.sina.com.cn/stock/usstock/c/2023-08-17/doc-imzhmvhi6819724.shtml'},
 {'news_title': '桥水看空美股美元',
  'news_url': 'https://finance.sina.com.cn/stock/usstock/c/2023-08-16/doc-imzhmksr3797616.shtml'},
 {'news_title': '美消费者穷途末路',
  'news_url': 'https://finance.sina.com.cn/stock/usstock/c/2023-08-17/doc-imzhnfwf3358825.shtml'},
 {'news_title': '美30年抵押贷款利率创2001年新高',
  'news_url': 'https://fi

## 使用OpenAI的API分析要闻情感

爬取到要闻的标题后，我们使用OpanAI提供的API来分析要闻标题的情感

调用OpenAI的API需要首先获得API_KEY，在登陆了OpenAI的账号后，前往`https://platform.openai.com/account/api-keys`页面即可生成，只需将下面的`your-api-key`部分换成自己的API_KEY即可

In [5]:
openai.api_key = "your-api-key"

接着定义了请求OpenAI API的函数，使用`gpt-3.5-turbo`模型

在`create`函数中的`messages`参数中，分别两个`prompt`，其中`role`为`system`的`prompt`代表我们对模型的指令，而`role`为`user`的`prompt`代表我们实际的问题；`temperature`的值为0代表期望模型的输出更加集中和确定

In [6]:
def judge_sentiment(news_title):
    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": '忘掉之前所有的指令。假装你是一位金融专家。你是一位有荐股经验的金融专家。如果是好消息，请回答 "积极"；如果是坏消息，请回答 "消极"；如果是不确定的消息，请回答 "未知"。然后用一个简短的句子详细说明。'},
            {"role": "user", "content": news_title},
        ],
        temperature=0,
    )
    return res["choices"][0]['message']['content']

接着我们将上面爬取到的要闻标题传入到`judge_sentiment`函数中，并且将返回的结果打印，另外由于免费的API_KEY在一分钟内只能发起三次调用，超出调用的阈值会直接导致代码异常，因此这里对异常进行处理，在达到阈值后暂停25秒钟，我们将前10个要闻标题进行情感分析

In [9]:
for item in data[:10]:
    news_title = item['news_title']
    try:
        print("新闻标题：%s" % news_title)
        print("情感分析结果：%s" % judge_sentiment(news_title))
    except openai.error.RateLimitError:
        sleep(25) # 超出阈值睡眠25秒
        print("情感分析结果：%s" % judge_sentiment(news_title))
    print("-" * 30)

新闻标题：经济日报:以暂停新股发行来救市不可取
情感分析结果：未知。暂停新股发行可能会对市场造成一定的影响，但具体效果还不确定。
------------------------------
新闻标题：股市转牛依靠印花税？机构：政策象征意义或大于资金面宽松意义
情感分析结果：未知。这个消息提到了股市转牛可能依靠印花税，但机构表示政策象征意义可能大于资金面宽松意义，因此无法确定股市的走势。
------------------------------
新闻标题：减免证券交易印花税呼声渐高！分析人士称具备调整必要性和空间
情感分析结果：积极。减免证券交易印花税的呼声越来越高，分析人士认为有必要和空间进行调整。这可能会降低投资者的交易成本，提升市场活跃度。
------------------------------
新闻标题：美联储会议纪要暗示加息或将继续
情感分析结果：消极。美联储会议纪要暗示加息或将继续，这可能会增加借贷成本，对经济和股市产生负面影响。
------------------------------
新闻标题：通胀风险成联储纪要关键词
情感分析结果：未知。联储纪要中提到通胀风险，但具体影响尚不确定。
------------------------------
新闻标题：桥水看空美股美元
情感分析结果：消极。桥水看空美股和美元意味着他们认为美国股市和美元将面临下跌的风险，可能会对投资者造成负面影响。
------------------------------
新闻标题：美消费者穷途末路
情感分析结果：消极。美消费者陷入困境可能导致经济增长放缓，股市下跌。
------------------------------
新闻标题：美30年抵押贷款利率创2001年新高
情感分析结果：消极。美国30年抵押贷款利率创下了自2001年以来的新高，这可能会导致房屋贷款成本上升，对房地产市场和经济产生负面影响。
------------------------------
新闻标题：美股大跌
情感分析结果：消极。美股的大跌可能意味着市场情绪疲软，投资者信心下降，可能会导致股市进一步下跌。
------------------------------
新闻标题：特斯拉蒸发1700亿元
情感分析结果：消极。特斯拉蒸发1700亿元意味着该公司的市值大幅下降，可能是由于投资